In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
import joblib

import talib as ta
from talib import MA_Type
import pickle


In [2]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [3]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.value = float(value)
        print("Total value of the account: ", self.value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [4]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

In [5]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:hfarm


NextValidId: 12


ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj


In [6]:
with open('portfolio_list.pickle', 'rb') as f:
    portfolio_list = pickle.load(f)

print(portfolio_list)

['ADP', 'AJG', 'KMX', 'DVN', 'FITB', 'LNC', 'LYV', 'TRGP', 'YUM', 'ZBH']


# query the current portfolio


In [7]:
app.reqAccountUpdates(True, "DU6730183") #TODO: replace the DU222526 with our account ID
# port_event.wait()
# port_event.clear()

time.sleep(10)
last_portfolio = app.last_portfolio
app.reqAccountUpdates(False, "DU6730183") 

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


In [8]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,AZO,40,2554.045410,102161.82,2450.035000,4160.42,0.0
1,BKNG,40,2396.592285,95863.69,2472.915000,-3052.91,0.0
2,CMG,60,1524.789917,91487.40,1695.016667,-10213.60,0.0
3,MKTX,264,349.734345,92329.87,382.975000,-8775.53,0.0
4,NOW,210,436.899445,91748.88,491.365000,-11437.77,0.0
5,NVR,18,5068.379883,91230.84,5468.345556,-7199.38,0.0
6,REGN,131,757.283142,99204.09,751.107634,808.99,0.0
7,SIVB,318,285.104065,90663.09,336.625000,-16383.66,0.0
8,TSLA,550,198.429993,109136.50,193.121364,2919.75,0.0
9,URI,219,441.135284,96608.63,461.835000,-4533.24,0.0


In [9]:
app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.value
app.cancelAccountSummary(9001)

Total value of the account:  947572.11
end of account summary


In [10]:
total_value

947572.11

# implementation
* decide what to buy & sell, as well as the quantity

In [11]:

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


Tick Price. Ticker Id: 0 tickType: 4 Price: 221.24
Tick Price. Ticker Id: 0 tickType: 6 Price: 224.04
Tick Price. Ticker Id: 0 tickType: 7 Price: 220.36
Tick Price. Ticker Id: 0 tickType: 9 Price: 222.93
Tick Price. Ticker Id: 0 tickType: 14 Price: 223.62
Tick Price. Ticker Id: 0 tickType: 1 Price: 221.07
Tick Price. Ticker Id: 0 tickType: 2 Price: 221.25
Tick Price. Ticker Id: 1 tickType: 4 Price: 184.55
Tick Price. Ticker Id: 1 tickType: 6 Price: 187.69
Tick Price. Ticker Id: 1 tickType: 7 Price: 184.26
Tick Price. Ticker Id: 1 tickType: 9 Price: 186.29
Tick Price. Ticker Id: 1 tickType: 14 Price: 187.46
Tick Price. Ticker Id: 1 tickType: 1 Price: 184.63
Tick Price. Ticker Id: 1 tickType: 2 Price: 184.92
Tick Price. Ticker Id: 2 tickType: 6 Price: 70.1
Tick Price. Ticker Id: 2 tickType: 7 Price: 67.56
Tick Price. Ticker Id: 2 tickType: 9 Price: 69.49
Tick Price. Ticker Id: 2 tickType: 4 Price: 68.24
Tick Price. Ticker Id: 2 tickType: 14 Price: 69.8
Tick Price. Ticker Id: 2 tickType: 

In [12]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [13]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value * 0.9) / target_portfolio['mkt_price'])

In [14]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,AZO,40,2554.045410,102161.82,2450.035000,4160.42,0.0
1,BKNG,40,2396.592285,95863.69,2472.915000,-3052.91,0.0
2,CMG,60,1524.789917,91487.40,1695.016667,-10213.60,0.0
3,MKTX,264,349.734345,92329.87,382.975000,-8775.53,0.0
4,NOW,210,436.899445,91748.88,491.365000,-11437.77,0.0
5,NVR,18,5068.379883,91230.84,5468.345556,-7199.38,0.0
6,REGN,131,757.283142,99204.09,751.107634,808.99,0.0
7,SIVB,318,285.104065,90663.09,336.625000,-16383.66,0.0
8,TSLA,550,198.429993,109136.50,193.121364,2919.75,0.0
9,URI,219,441.135284,96608.63,461.835000,-4533.24,0.0


In [15]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,ADP,0.1,222.93,382.0
1,AJG,0.1,186.29,457.0
2,KMX,0.1,69.49,1227.0
3,DVN,0.1,53.59,1591.0
4,FITB,0.1,35.93,2373.0
5,LNC,0.1,32.32,2638.0
6,LYV,0.1,75.44,1130.0
7,TRGP,0.1,75.56,1128.0
8,YUM,0.1,129.31,659.0
9,ZBH,0.1,123.62,689.0


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [16]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = last_portfolio_stock['quantity'].values[0]
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['ticker'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['ticker'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [17]:
stocks_to_buy

[['LYV', 1130.0],
 ['ADP', 382.0],
 ['TRGP', 1128.0],
 ['AJG', 457.0],
 ['DVN', 1591.0],
 ['YUM', 659.0],
 ['LNC', 2638.0],
 ['KMX', 1227.0],
 ['ZBH', 689.0],
 ['FITB', 2373.0]]

In [18]:
stocks_to_sell

[['CMG', Decimal('60')],
 ['URI', Decimal('219')],
 ['MKTX', Decimal('264')],
 ['NVR', Decimal('18')],
 ['AZO', Decimal('40')],
 ['REGN', Decimal('131')],
 ['NOW', Decimal('210')],
 ['BKNG', Decimal('40')],
 ['TSLA', Decimal('550')],
 ['SIVB', Decimal('318')]]

# place order

In [19]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

count = 0
for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
    app.reqAccountSummary(9001, "All", "SettledCash")
    value_event.wait()
    value_event.clear()
    cash_value = app.value
    if count >= len(stocks_to_buy)/2 and cash_value <= 0:
        break # stop buying when cash is below 0
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()
    count +=1

NextValidId: 12
openOrder. orderId:13, contract:37655664,CMG,STK,,0,?,,SMART,,USD,CMG,CMG,False,,,,combo:, order:13,1000,998747787: MKT SELL 60@1523.85 GTC
orderStatus. orderId: 13, status: Submitted, filled: 0, remaining:60, avgFillPrice: 0.0, permId:998747787, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:14, contract:3629755,URI,STK,,0,?,,SMART,,USD,URI,URI,False,,,,combo:, order:14,1000,998747788: MKT SELL 219@440.93 GTC
orderStatus. orderId: 14, status: Submitted, filled: 0, remaining:219, avgFillPrice: 0.0, permId:998747788, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
execDetails. reqId: -1, contract: 3629755,URI,STK,,0,,,CHX,,USD,URI,URI,False,,,,combo:, execution: ExecId: 00025b46.63f793d6.01.01, Time: 20230223 13:21:49 US/Eastern, Account: DU6730183, Exchange: CHX, Side: SLD, Shares: 19, Price: 440.92, PermId: 998747788, ClientId: 1000, OrderId: 14, Liquidation: 0, CumQty: 19, AvgPrice: 440.92, Orde

openOrder. orderId:32, contract:269318,FITB,STK,,0,?,,SMART,,USD,FITB,NMS,False,,,,combo:, order:32,1000,998747806: MKT BUY 2373@0 GTC
orderStatus. orderId: 32, status: PreSubmitted, filled: 0, remaining:2373, avgFillPrice: 0.0, permId:998747806, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
execDetails. reqId: -1, contract: 269318,FITB,STK,,0,,,PEARL,,USD,FITB,NMS,False,,,,combo:, execution: ExecId: 00025b44.63f768df.01.01, Time: 20230223 13:21:51 US/Eastern, Account: DU6730183, Exchange: PEARL, Side: BOT, Shares: 100, Price: 35.8, PermId: 998747806, ClientId: 1000, OrderId: 32, Liquidation: 0, CumQty: 100, AvgPrice: 35.8, OrderRef: , EvRule: , EvMultiplier: 0, ModelCode: , LastLiquidity: 2
openOrder. orderId:32, contract:269318,FITB,STK,,0,?,,SMART,,USD,FITB,NMS,False,,,,combo:, order:32,1000,998747806: MKT BUY 2373@0 GTC
orderStatus. orderId: 32, status: PreSubmitted, filled: 100, remaining:2273, avgFillPrice: 35.8, permId:998747806, parentId:0, lastFill

ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj
